In [6]:
'''ran once, to create the weather prediction data from earlier fetched 3 months datasets, now commented out

import pandas as pd

weather_pred_past = pd.read_csv('../data/weather/Weather_pred_past_3m_DeBilt_2025-03-25.csv')
print(weather_pred_past.head())
weather_pred_past['date'] = pd.to_datetime(weather_pred_past['date'])

print(f"Date range: {weather_pred_past['date'].min()} to {weather_pred_past['date'].max()}")'''


'ran once, to create the weather prediction data from earlier fetched 3 months datasets, now commented out\n\nimport pandas as pd\n\nweather_pred_past = pd.read_csv(\'../data/weather/Weather_pred_past_3m_DeBilt_2025-03-25.csv\')\nprint(weather_pred_past.head())\nweather_pred_past[\'date\'] = pd.to_datetime(weather_pred_past[\'date\'])\n\nprint(f"Date range: {weather_pred_past[\'date\'].min()} to {weather_pred_past[\'date\'].max()}")'

In [7]:
'''import sqlite3

# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Write the DataFrame to the database table 'raw_weather_preds'
# If table exists, replace it. If not, create new table
weather_pred_past.to_sql('raw_weather_preds', conn, if_exists='replace', index=False)


record_count = weather_pred_past.shape[0]
print(f"{record_count} records successfully written to database table 'raw_weather_preds'")
# Close the connection
conn.close()
'''


'import sqlite3\n\n# Connect to the SQLite database\ndb_path = \'../data/WARP.db\'\nconn = sqlite3.connect(db_path)\n\n# Write the DataFrame to the database table \'raw_weather_preds\'\n# If table exists, replace it. If not, create new table\nweather_pred_past.to_sql(\'raw_weather_preds\', conn, if_exists=\'replace\', index=False)\n\n\nrecord_count = weather_pred_past.shape[0]\nprint(f"{record_count} records successfully written to database table \'raw_weather_preds\'")\n# Close the connection\nconn.close()\n'

In [1]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Connect to the SQLite database using the existing db_path
conn = sqlite3.connect(db_path)
# Query the last 5 rows from raw_weather_preds table
df_head = pd.read_sql_query("SELECT * FROM raw_weather_preds ORDER BY date DESC LIMIT 5", conn)

# Close the connection
conn.close()

# Display the results
print(df_head)

# Connect to the SQLite database
# collect the date column from the raw_weather_preds table
conn = sqlite3.connect(db_path)
hist_weather_dates = pd.read_sql_query("SELECT date FROM raw_weather_preds", conn)
conn.close()

# Convert to datetime and find max date
hist_weather_dates['date'] = pd.to_datetime(hist_weather_dates['date'])
most_recent_weather_pred_date = hist_weather_dates['date'].max().strftime('%Y-%m-%d')
print(f"The most recent date in raw_weather_preds is: {most_recent_weather_pred_date}")

                        date  temperature_2m temperature_2m_previous_day1  \
0  2025-05-15 23:00:00+00:00       10.063000                         None   
1  2025-05-15 22:00:00+00:00       10.813000                         None   
2  2025-05-15 21:00:00+00:00       11.813000                         None   
3  2025-05-15 20:00:00+00:00       12.912999                         None   
4  2025-05-15 19:00:00+00:00       14.162999                         None   

  temperature_2m_previous_day2 temperature_2m_previous_day3  \
0                         None                         None   
1                         None                         None   
2                         None                         None   
3                         None                         None   
4                         None                         None   

  temperature_2m_previous_day4 temperature_2m_previous_day5  \
0                         None                         None   
1                         None  

In [2]:
# De Bilt up to 3 months in the past, to append the data to the database
from datetime import datetime, timezone
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# define now
now = datetime.now(timezone.utc)
# Convert current time to YYYY-MM-DD format
now_date_str = now.strftime("%Y-%m-%d")

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://previous-runs-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.108499,
	"longitude": 5.180616,
	"hourly": ["temperature_2m",
    "temperature_2m_previous_day1","temperature_2m_previous_day2",
    "temperature_2m_previous_day3","temperature_2m_previous_day4",
    "temperature_2m_previous_day5","temperature_2m_previous_day6",
    "temperature_2m_previous_day7",
    "wind_speed_10m","wind_speed_10m_previous_day1",
    "wind_speed_10m_previous_day2","wind_speed_10m_previous_day3",
    "wind_speed_10m_previous_day4","wind_speed_10m_previous_day5",
    "wind_speed_10m_previous_day6","wind_speed_10m_previous_day7",
    "wind_direction_10m_previous_day7","wind_direction_10m_previous_day6","wind_direction_10m_previous_day5",
    "cloud_cover","cloud_cover_previous_day1",
    "cloud_cover_previous_day2","cloud_cover_previous_day3",
    "cloud_cover_previous_day4","cloud_cover_previous_day5",
    "cloud_cover_previous_day6","cloud_cover_previous_day7",
    "snowfall","snowfall_previous_day1",
    "snowfall_previous_day2","snowfall_previous_day3",
    "snowfall_previous_day4","snowfall_previous_day5",
    "snowfall_previous_day6","snowfall_previous_day7",
    "apparent_temperature","apparent_temperature_previous_day1",
    "apparent_temperature_previous_day2","apparent_temperature_previous_day3",
    "apparent_temperature_previous_day4","apparent_temperature_previous_day5",
    "apparent_temperature_previous_day6","apparent_temperature_previous_day7",
    "wind_direction_10m_previous_day4","wind_direction_10m_previous_day3",
    "wind_direction_10m_previous_day2","wind_direction_10m_previous_day1",
    "wind_direction_10m",
    "diffuse_radiation","diffuse_radiation_previous_day1",
    "diffuse_radiation_previous_day2","diffuse_radiation_previous_day3",
    "diffuse_radiation_previous_day4","diffuse_radiation_previous_day5",
    "diffuse_radiation_previous_day6","diffuse_radiation_previous_day7",
    "direct_normal_irradiance","direct_normal_irradiance_previous_day1",
    "direct_normal_irradiance_previous_day2","direct_normal_irradiance_previous_day3","direct_normal_irradiance_previous_day4",
    "direct_normal_irradiance_previous_day5","direct_normal_irradiance_previous_day6",
    "direct_normal_irradiance_previous_day7",
    "shortwave_radiation","shortwave_radiation_previous_day1",
    "shortwave_radiation_previous_day2","shortwave_radiation_previous_day3",
    "shortwave_radiation_previous_day4","shortwave_radiation_previous_day5",
    "shortwave_radiation_previous_day6","shortwave_radiation_previous_day7",
    "direct_radiation","direct_radiation_previous_day1",
    "direct_radiation_previous_day2","direct_radiation_previous_day3",
    "direct_radiation_previous_day4","direct_radiation_previous_day5",
    "direct_radiation_previous_day6","direct_radiation_previous_day7"],
"models": "knmi_seamless",
	"start_date": most_recent_weather_pred_date,
    "end_date": now_date_str,
}
responses = openmeteo.weather_api(url, params=params)

# Process single location. (would have to add a for-loop for multiple locations or weather models)
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

							# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_temperature_2m_previous_day1 = hourly.Variables(1).ValuesAsNumpy()
hourly_temperature_2m_previous_day2 = hourly.Variables(2).ValuesAsNumpy()
hourly_temperature_2m_previous_day3 = hourly.Variables(3).ValuesAsNumpy()
hourly_temperature_2m_previous_day4 = hourly.Variables(4).ValuesAsNumpy()
hourly_temperature_2m_previous_day5 = hourly.Variables(5).ValuesAsNumpy()
hourly_temperature_2m_previous_day6 = hourly.Variables(6).ValuesAsNumpy()
hourly_temperature_2m_previous_day7 = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(8).ValuesAsNumpy()
hourly_wind_speed_10m_previous_day1 = hourly.Variables(9).ValuesAsNumpy()
hourly_wind_speed_10m_previous_day2 = hourly.Variables(10).ValuesAsNumpy()
hourly_wind_speed_10m_previous_day3 = hourly.Variables(11).ValuesAsNumpy()
hourly_wind_speed_10m_previous_day4 = hourly.Variables(12).ValuesAsNumpy()
hourly_wind_speed_10m_previous_day5 = hourly.Variables(13).ValuesAsNumpy()
hourly_wind_speed_10m_previous_day6 = hourly.Variables(14).ValuesAsNumpy()
hourly_wind_speed_10m_previous_day7 = hourly.Variables(15).ValuesAsNumpy()
hourly_wind_direction_10m_previous_day7 = hourly.Variables(16).ValuesAsNumpy()
hourly_wind_direction_10m_previous_day6 = hourly.Variables(17).ValuesAsNumpy()
hourly_wind_direction_10m_previous_day5 = hourly.Variables(18).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(19).ValuesAsNumpy()
hourly_cloud_cover_previous_day1 = hourly.Variables(20).ValuesAsNumpy()
hourly_cloud_cover_previous_day2 = hourly.Variables(21).ValuesAsNumpy()
hourly_cloud_cover_previous_day3 = hourly.Variables(22).ValuesAsNumpy()
hourly_cloud_cover_previous_day4 = hourly.Variables(23).ValuesAsNumpy()
hourly_cloud_cover_previous_day5 = hourly.Variables(24).ValuesAsNumpy()
hourly_cloud_cover_previous_day6 = hourly.Variables(25).ValuesAsNumpy()
hourly_cloud_cover_previous_day7 = hourly.Variables(26).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(27).ValuesAsNumpy()
hourly_snowfall_previous_day1 = hourly.Variables(28).ValuesAsNumpy()
hourly_snowfall_previous_day2 = hourly.Variables(29).ValuesAsNumpy()
hourly_snowfall_previous_day3 = hourly.Variables(30).ValuesAsNumpy()
hourly_snowfall_previous_day4 = hourly.Variables(31).ValuesAsNumpy()
hourly_snowfall_previous_day5 = hourly.Variables(32).ValuesAsNumpy()
hourly_snowfall_previous_day6 = hourly.Variables(33).ValuesAsNumpy()
hourly_snowfall_previous_day7 = hourly.Variables(34).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(35).ValuesAsNumpy()
hourly_apparent_temperature_previous_day1 = hourly.Variables(36).ValuesAsNumpy()
hourly_apparent_temperature_previous_day2 = hourly.Variables(37).ValuesAsNumpy()
hourly_apparent_temperature_previous_day3 = hourly.Variables(38).ValuesAsNumpy()
hourly_apparent_temperature_previous_day4 = hourly.Variables(39).ValuesAsNumpy()
hourly_apparent_temperature_previous_day5 = hourly.Variables(40).ValuesAsNumpy()
hourly_apparent_temperature_previous_day6 = hourly.Variables(41).ValuesAsNumpy()
hourly_apparent_temperature_previous_day7 = hourly.Variables(42).ValuesAsNumpy()
hourly_wind_direction_10m_previous_day4 = hourly.Variables(43).ValuesAsNumpy()
hourly_wind_direction_10m_previous_day3 = hourly.Variables(44).ValuesAsNumpy()
hourly_wind_direction_10m_previous_day2 = hourly.Variables(45).ValuesAsNumpy()
hourly_wind_direction_10m_previous_day1 = hourly.Variables(46).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(47).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(48).ValuesAsNumpy()
hourly_diffuse_radiation_previous_day1 = hourly.Variables(49).ValuesAsNumpy()
hourly_diffuse_radiation_previous_day2 = hourly.Variables(50).ValuesAsNumpy()
hourly_diffuse_radiation_previous_day3 = hourly.Variables(51).ValuesAsNumpy()
hourly_diffuse_radiation_previous_day4 = hourly.Variables(52).ValuesAsNumpy()
hourly_diffuse_radiation_previous_day5 = hourly.Variables(53).ValuesAsNumpy()
hourly_diffuse_radiation_previous_day6 = hourly.Variables(54).ValuesAsNumpy()
hourly_diffuse_radiation_previous_day7 = hourly.Variables(55).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(56).ValuesAsNumpy()
hourly_direct_normal_irradiance_previous_day1 = hourly.Variables(57).ValuesAsNumpy()
hourly_direct_normal_irradiance_previous_day2 = hourly.Variables(58).ValuesAsNumpy()
hourly_direct_normal_irradiance_previous_day3 = hourly.Variables(59).ValuesAsNumpy()
hourly_direct_normal_irradiance_previous_day4 = hourly.Variables(60).ValuesAsNumpy()
hourly_direct_normal_irradiance_previous_day5 = hourly.Variables(61).ValuesAsNumpy()
hourly_direct_normal_irradiance_previous_day6 = hourly.Variables(62).ValuesAsNumpy()
hourly_direct_normal_irradiance_previous_day7 = hourly.Variables(63).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(64).ValuesAsNumpy()
hourly_shortwave_radiation_previous_day1 = hourly.Variables(65).ValuesAsNumpy()
hourly_shortwave_radiation_previous_day2 = hourly.Variables(66).ValuesAsNumpy()
hourly_shortwave_radiation_previous_day3 = hourly.Variables(67).ValuesAsNumpy()
hourly_shortwave_radiation_previous_day4 = hourly.Variables(68).ValuesAsNumpy()
hourly_shortwave_radiation_previous_day5 = hourly.Variables(69).ValuesAsNumpy()
hourly_shortwave_radiation_previous_day6 = hourly.Variables(70).ValuesAsNumpy()
hourly_shortwave_radiation_previous_day7 = hourly.Variables(71).ValuesAsNumpy()
hourly_direct_radiation = hourly.Variables(72).ValuesAsNumpy()
hourly_direct_radiation_previous_day1 = hourly.Variables(73).ValuesAsNumpy()
hourly_direct_radiation_previous_day2 = hourly.Variables(74).ValuesAsNumpy()
hourly_direct_radiation_previous_day3 = hourly.Variables(75).ValuesAsNumpy()
hourly_direct_radiation_previous_day4 = hourly.Variables(76).ValuesAsNumpy()
hourly_direct_radiation_previous_day5 = hourly.Variables(77).ValuesAsNumpy()
hourly_direct_radiation_previous_day6 = hourly.Variables(78).ValuesAsNumpy()
hourly_direct_radiation_previous_day7 = hourly.Variables(79).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["temperature_2m_previous_day1"] = hourly_temperature_2m_previous_day1
hourly_data["temperature_2m_previous_day2"] = hourly_temperature_2m_previous_day2
hourly_data["temperature_2m_previous_day3"] = hourly_temperature_2m_previous_day3
hourly_data["temperature_2m_previous_day4"] = hourly_temperature_2m_previous_day4
hourly_data["temperature_2m_previous_day5"] = hourly_temperature_2m_previous_day5
hourly_data["temperature_2m_previous_day6"] = hourly_temperature_2m_previous_day6
hourly_data["temperature_2m_previous_day7"] = hourly_temperature_2m_previous_day7
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_10m_previous_day1"] = hourly_wind_speed_10m_previous_day1
hourly_data["wind_speed_10m_previous_day2"] = hourly_wind_speed_10m_previous_day2
hourly_data["wind_speed_10m_previous_day3"] = hourly_wind_speed_10m_previous_day3
hourly_data["wind_speed_10m_previous_day4"] = hourly_wind_speed_10m_previous_day4
hourly_data["wind_speed_10m_previous_day5"] = hourly_wind_speed_10m_previous_day5
hourly_data["wind_speed_10m_previous_day6"] = hourly_wind_speed_10m_previous_day6
hourly_data["wind_speed_10m_previous_day7"] = hourly_wind_speed_10m_previous_day7
hourly_data["wind_direction_10m_previous_day7"] = hourly_wind_direction_10m_previous_day7
hourly_data["wind_direction_10m_previous_day6"] = hourly_wind_direction_10m_previous_day6
hourly_data["wind_direction_10m_previous_day5"] = hourly_wind_direction_10m_previous_day5
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_previous_day1"] = hourly_cloud_cover_previous_day1
hourly_data["cloud_cover_previous_day2"] = hourly_cloud_cover_previous_day2
hourly_data["cloud_cover_previous_day3"] = hourly_cloud_cover_previous_day3
hourly_data["cloud_cover_previous_day4"] = hourly_cloud_cover_previous_day4
hourly_data["cloud_cover_previous_day5"] = hourly_cloud_cover_previous_day5
hourly_data["cloud_cover_previous_day6"] = hourly_cloud_cover_previous_day6
hourly_data["cloud_cover_previous_day7"] = hourly_cloud_cover_previous_day7
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snowfall_previous_day1"] = hourly_snowfall_previous_day1
hourly_data["snowfall_previous_day2"] = hourly_snowfall_previous_day2
hourly_data["snowfall_previous_day3"] = hourly_snowfall_previous_day3
hourly_data["snowfall_previous_day4"] = hourly_snowfall_previous_day4
hourly_data["snowfall_previous_day5"] = hourly_snowfall_previous_day5
hourly_data["snowfall_previous_day6"] = hourly_snowfall_previous_day6
hourly_data["snowfall_previous_day7"] = hourly_snowfall_previous_day7
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["apparent_temperature_previous_day1"] = hourly_apparent_temperature_previous_day1
hourly_data["apparent_temperature_previous_day2"] = hourly_apparent_temperature_previous_day2
hourly_data["apparent_temperature_previous_day3"] = hourly_apparent_temperature_previous_day3
hourly_data["apparent_temperature_previous_day4"] = hourly_apparent_temperature_previous_day4
hourly_data["apparent_temperature_previous_day5"] = hourly_apparent_temperature_previous_day5
hourly_data["apparent_temperature_previous_day6"] = hourly_apparent_temperature_previous_day6
hourly_data["apparent_temperature_previous_day7"] = hourly_apparent_temperature_previous_day7
hourly_data["wind_direction_10m_previous_day4"] = hourly_wind_direction_10m_previous_day4
hourly_data["wind_direction_10m_previous_day3"] = hourly_wind_direction_10m_previous_day3
hourly_data["wind_direction_10m_previous_day2"] = hourly_wind_direction_10m_previous_day2
hourly_data["wind_direction_10m_previous_day1"] = hourly_wind_direction_10m_previous_day1
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
hourly_data["diffuse_radiation_previous_day1"] = hourly_diffuse_radiation_previous_day1
hourly_data["diffuse_radiation_previous_day2"] = hourly_diffuse_radiation_previous_day2
hourly_data["diffuse_radiation_previous_day3"] = hourly_diffuse_radiation_previous_day3
hourly_data["diffuse_radiation_previous_day4"] = hourly_diffuse_radiation_previous_day4
hourly_data["diffuse_radiation_previous_day5"] = hourly_diffuse_radiation_previous_day5
hourly_data["diffuse_radiation_previous_day6"] = hourly_diffuse_radiation_previous_day6
hourly_data["diffuse_radiation_previous_day7"] = hourly_diffuse_radiation_previous_day7
hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
hourly_data["direct_normal_irradiance_previous_day1"] = hourly_direct_normal_irradiance_previous_day1
hourly_data["direct_normal_irradiance_previous_day2"] = hourly_direct_normal_irradiance_previous_day2
hourly_data["direct_normal_irradiance_previous_day3"] = hourly_direct_normal_irradiance_previous_day3
hourly_data["direct_normal_irradiance_previous_day4"] = hourly_direct_normal_irradiance_previous_day4
hourly_data["direct_normal_irradiance_previous_day5"] = hourly_direct_normal_irradiance_previous_day5
hourly_data["direct_normal_irradiance_previous_day6"] = hourly_direct_normal_irradiance_previous_day6
hourly_data["direct_normal_irradiance_previous_day7"] = hourly_direct_normal_irradiance_previous_day7
hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
hourly_data["shortwave_radiation_previous_day1"] = hourly_shortwave_radiation_previous_day1
hourly_data["shortwave_radiation_previous_day2"] = hourly_shortwave_radiation_previous_day2
hourly_data["shortwave_radiation_previous_day3"] = hourly_shortwave_radiation_previous_day3
hourly_data["shortwave_radiation_previous_day4"] = hourly_shortwave_radiation_previous_day4
hourly_data["shortwave_radiation_previous_day5"] = hourly_shortwave_radiation_previous_day5
hourly_data["shortwave_radiation_previous_day6"] = hourly_shortwave_radiation_previous_day6
hourly_data["shortwave_radiation_previous_day7"] = hourly_shortwave_radiation_previous_day7
hourly_data["direct_radiation"] = hourly_direct_radiation
hourly_data["direct_radiation_previous_day1"] = hourly_direct_radiation_previous_day1
hourly_data["direct_radiation_previous_day2"] = hourly_direct_radiation_previous_day2
hourly_data["direct_radiation_previous_day3"] = hourly_direct_radiation_previous_day3
hourly_data["direct_radiation_previous_day4"] = hourly_direct_radiation_previous_day4
hourly_data["direct_radiation_previous_day5"] = hourly_direct_radiation_previous_day5
hourly_data["direct_radiation_previous_day6"] = hourly_direct_radiation_previous_day6
hourly_data["direct_radiation_previous_day7"] = hourly_direct_radiation_previous_day7

recent_pred_dataframe = pd.DataFrame(data = hourly_data)
print(recent_pred_dataframe)


Coordinates 52.11399841308594°N 5.190999984741211°E
Elevation 6.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s
                         date  temperature_2m  temperature_2m_previous_day1  \
0   2025-05-15 00:00:00+00:00       10.162999                     10.212999   
1   2025-05-15 01:00:00+00:00       10.212999                      9.912999   
2   2025-05-15 02:00:00+00:00       11.063000                     10.212999   
3   2025-05-15 03:00:00+00:00       10.863000                     10.563000   
4   2025-05-15 04:00:00+00:00       10.613000                     10.113000   
..                        ...             ...                           ...   
115 2025-05-19 19:00:00+00:00       17.963001                     17.913000   
116 2025-05-19 20:00:00+00:00       17.163000                     16.413000   
117 2025-05-19 21:00:00+00:00       17.263000                     15.263000   
118 2025-05-19 22:00:00+00:00       15.813000                     14.013000   
119 2025

In [3]:
conn = sqlite3.connect(db_path)

# Read existing data from raw_weather_preds
existing_data = pd.read_sql_query("SELECT * FROM raw_weather_preds", conn)

# Convert date columns to datetime for both dataframes
existing_data['date'] = pd.to_datetime(existing_data['date'])
recent_pred_dataframe['date'] = pd.to_datetime(recent_pred_dataframe['date'])

# Remove any duplicates based on date and keep the latest values
merged_df = pd.concat([existing_data, recent_pred_dataframe])
initial_rows = len(merged_df)
merged_df = merged_df.drop_duplicates(subset='date', keep='last')
removed_rows = initial_rows - len(merged_df)
print(f"Removed {removed_rows} duplicate rows")

# Sort by date
merged_df = merged_df.sort_values('date')

# Write the merged dataframe back to the database
merged_df.to_sql('raw_weather_preds', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print(f"Data successfully merged. Total rows: {len(merged_df)}")
print(f"Date range: {merged_df['date'].min()} to {merged_df['date'].max()}")

Removed 24 duplicate rows
Data successfully merged. Total rows: 3504
Date range: 2024-12-25 00:00:00+00:00 to 2025-05-19 23:00:00+00:00
